In [ ]:
import numpy as np
import sys
import casadi

# Import do_mpc package:
import do_mpc

In [ ]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)